## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-12-05-20-23 +0000,nypost,"US soccer legend Abby Wambach dumps agency, ca...",https://nypost.com/2026/02/12/us-news/us-socce...
1,2026-02-12-05-15-44 +0000,nyt,India’s Prime Minister Faces Blowback Over Tra...,https://www.nytimes.com/2026/02/12/business/mo...
2,2026-02-12-05-01-09 +0000,nyt,Olympic Figure Skaters Are on Thin Ice Over Mu...,https://www.nytimes.com/2026/02/12/world/europ...
3,2026-02-12-05-01-08 +0000,nyt,Turkish Nobel Novelist Orhan Pamuk Gets the Ne...,https://www.nytimes.com/2026/02/12/world/europ...
4,2026-02-12-05-00-00 +0000,wsj,Origin Energy Sees Opportunity for Data Center...,https://www.wsj.com/business/energy-oil/origin...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2420/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
19,trump,42
179,canada,20
778,new,17
185,shooting,16
25,ice,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
281,2026-02-11-15-19-00 +0000,wsj,House lawmakers rejected an attempt by Speaker...,https://www.wsj.com/politics/policy/house-reje...,148
68,2026-02-12-00-44-00 +0000,wsj,The GOP-led House passed a resolution designed...,https://www.wsj.com/politics/policy/gop-led-ho...,127
91,2026-02-11-23-37-50 +0000,nypost,Six House Republicans break with White House t...,https://nypost.com/2026/02/11/us-news/six-hous...,106
297,2026-02-11-14-03-29 +0000,cbc,U.S. House votes against Trump's tariffs on Ca...,https://www.cbc.ca/news/world/trump-tariffs-ca...,94
317,2026-02-11-11-57-00 +0000,wsj,Commerce Secretary Howard Lutnick became the f...,https://www.wsj.com/politics/policy/lutnicks-f...,93


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
281,148,2026-02-11-15-19-00 +0000,wsj,House lawmakers rejected an attempt by Speaker...,https://www.wsj.com/politics/policy/house-reje...
360,81,2026-02-11-09-03-03 +0000,nypost,‘Gunperson’ in a ‘dress’ behind Canada’s deadl...,https://nypost.com/2026/02/11/world-news/tumbl...
242,70,2026-02-11-17-27-07 +0000,nypost,Ex-Google CEO Eric Schmidt spent New Year’s wi...,https://nypost.com/2026/02/11/business/ex-goog...
148,55,2026-02-11-21-42-00 +0000,wsj,"At a congressional hearing, Attorney General P...",https://www.wsj.com/politics/policy/pam-bondi-...
213,47,2026-02-11-18-56-22 +0000,latimes,Warner Bros. Discovery board faces pressure as...,https://www.latimes.com/entertainment-arts/bus...
41,46,2026-02-12-02-31-36 +0000,nypost,Karen Bass ordered LA cops to turn on body cam...,https://nypost.com/2026/02/11/us-news/karen-ba...
110,45,2026-02-11-22-51-00 +0000,wsj,Federal Reserve governor Stephen Miran said he...,https://www.wsj.com/economy/central-banking/fe...
85,39,2026-02-11-23-49-00 +0000,wsj,The Republican-led House passed legislation th...,https://www.wsj.com/politics/elections/save-ac...
51,38,2026-02-12-02-02-00 +0000,wsj,Demand for healthcare workers is outstripping ...,https://www.wsj.com/economy/jobs/healthcare-jo...
79,38,2026-02-12-00-01-41 +0000,nypost,Mother killed by transgender Canadian school s...,https://nypost.com/2026/02/11/world-news/mothe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
